# Landscape evolution code for Hw 11

In [1]:
## Import necessary packages

%matplotlib osx 
# On Macs use osx
# For Windows use qt

import numpy as np
from numpy.random import rand
from landscapeWithOcean import LandscapeWithOcean # Import methods from inside file landscapeWithOcean.py

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm

In [2]:
def AddHill(Z,NX,NY,xx,yy,r,h):

    for x in range(NX):
        for y in range(NY):
            dx = np.mod(x-xx+NX/2,NX)-NX/2; # difference i-i0 but apply p.b.c. 
            dy = np.mod(y-yy+NY/2,NY)-NY/2;
            dr = np.sqrt(dx**2+dy**2);
            if (dr<r):
                Z[x,y] += h * (np.cos(dr/r*np.pi/2.0))**2;

    return Z

In [3]:
### Define simulation grid and initial conditions

NX = 10#70*2 #number of rows
NY = 10 #70*2 #number of columns

d  = 5 # grid spacing in meters
dx = d # keep dx=dy for simplicity
dy = d

LX=NX*dx
LY=NY*dy

# small random features in topography to begin erosion
Z = rand(NX,NY)*0.1    #Create an array of the given shape and populate it with random samples from a uniform distribution over [0, 1).
# for i in range(5):
#     xx = rand()*NX  #random number of rows
#     yy = rand()*NY   #random number of columns
#     r  = (0.1+rand())*NX  #random fraction of rows
#     h  = (0.1+rand())*15  #random 
#     Z = AddHill(Z,NX,NY,xx,yy,r,h)

    
for i in range(5):
    xx = 10*NX  #random number of rows
    yy = 10*NY   #random number of columns
    r  = 10#(0.1+rand())*NX  #random fraction of rows
    h  = (0.1+rand())*100  #random 
    Z = AddHill(Z,NX,NY,xx,yy,r,h)
    
for i in range(5):
    xx = 10*NX  #random number of rows
    yy = 10*NY   #random number of columns
    r  = 10#(0.1+rand())*NX  #random fraction of rows
    h  = (0.1+rand())*50  #random 
    Z = AddHill(Z,NX,NY,xx,yy,r,h)


# for i in range(5):
#     xx = rand()*NX
#     yy = rand()*NY
#     r  = (0.1+rand())*NX/2
#     h  = (0.5+rand())*10
#     Z = AddHill(Z,NX,NY,xx,yy,r,h)

x = np.arange(NX)
y = np.arange(NY)
X,Y = np.meshgrid(y,x) #strange that y goes first !!!
ZMaxOrg = np.max(Z)
#print(ZMaxOrg)

In [4]:
### Physical Parameters
K = 1.0e-6 # meters^(1-2m)/yr

D = 0.005 # m^2/yr

# uplift rate
# uplift = 0.03 / 600.
uplift = 0.0

In [5]:
### Model parameters

# Time step
dt = d**2 / D / 2. 
#dt = d**2 / D /16. #extra small steps 
print(' dt[years] = ',dt)

#Area exponent A^m, default m=1
m=1

#gradient exponent g^n, default n=1
n = 1

#erosion threshold 
theta_c = 10 

 dt[years] =  2500.0


In [6]:
# Total simulation time
T = 1000.0 * 625.0

# total number of iterations
n_iter = int(np.round(T/dt))
print('Number of interation: ',n_iter)

Number of interation:  250


In [7]:
# Initialize landscape 
ls = LandscapeWithOcean(NX,NY)  #calling the imported py file, takes number of rows and columns

oceanLevelParameter=0.2  # what does this parameter do?
ls.ComputeOceanVolumeFromOceanLevelParameter(Z,NX,NY,oceanLevelParameter)

ls.pool_check(Z,NX,NY)
ls.A = np.zeros((NX,NY))

Minimum elevation           108.88920106074137
Maximum elevation           552.0280720989576
Beach level                 197.5169752683846
Ocean volume                245.54178938966677
Percentage of ocean surface 5.0


In [8]:
# Set-up figure
def init_figure():
    fig = plt.figure(figsize=(12.,6.))
    plt.show()
    return fig

def update_figure():
        plt.clf()
        ax1 = fig.add_subplot(121,projection='3d')

        # use equal x-y aspect with an explicit vertical exageration
        vert_exag = 4.
        ax1.set_xlim3d(0,max(NX,NY))
        ax1.set_ylim3d(0,max(NX,NY))
        ax1.set_zlim3d(0,ZMaxOrg)

        ax1.set_title('Surface Relief')

#        surf = ax1.plot_surface(X,Y,Z, cmap = cm.terrain, rstride=1, cstride=1,
#                antialiased=False,linewidth=0)

        ZPlot = np.copy(Z)
        ZPlot[ZPlot<ls.ZBeachLevel] = ls.ZBeachLevel 
        ZPlot -= ls.ZBeachLevel
        ax1.plot_surface(X,Y,ZPlot, cmap = cm.terrain, rstride=1, cstride=1,
                antialiased=False,linewidth=0)

        ax2 = fig.add_subplot(122,aspect='equal')
        ax2.set_title('Elevation')

        #im = ax2.pcolor(Z,cmap=cm.terrain)
        im = ax2.pcolor(ZPlot,cmap=cm.coolwarm)
        cs = ax2.contour(ZPlot,6,colors='k')

        # Add a color bar which maps values to colors.
        cbar = fig.colorbar(im, shrink=0.5, aspect=5)
        # Add the contour line levels to the colorbar
        cbar.add_lines(cs)

        #plt.show()
        plt.draw()
        plt.pause(0.0001)

In [9]:
# Set up figure
fig = init_figure()
update_figure()
Znew = np.copy(Z)

for it in range(1,n_iter+1):
    
    ls.calculate_collection_area(Z,NX,NY)
    ls.A *= dx*dy
    
    for i in range(NX):
        iL = np.mod(i-1,NX) # normally i-1 but observe p.b.c.
        iR = np.mod(i+1,NX) # normally i+1 but observe p.b.c.

        for j in range(NY):
            jD = np.mod(j-1,NY) # normally j-1 but observe p.b.c.
            jU = np.mod(j+1,NY) # normally j+1 but observe p.b.c.
  
            if ls.ocean[i,j]>0:
                Psi_z  = 0;
                Phi_z  = 0;
            else:
                if ls.drain[i,j]>0: #this cell is a drain
                    s1 = (Z[iR,j]  - Z[iL,j] )/(2.*dx)
                    s2 = (Z[i,jU]  - Z[i,jD] )/(2.*dy)
                    s3 = (Z[iR,jD] - Z[iL,jU])/(2. * np.sqrt( dx**2 + dy**2) )
                    s4 = (Z[iR,jU] - Z[iL,jD])/(2. * np.sqrt( dx**2 + dy**2) )
                    gradient = (np.sqrt(s1**2 + s2**2) + np.sqrt(s3**2 + s4**2))/2.
                    Psi_z = K * ( ls.A[i,j]**m * gradient**n - theta_c)            

                elif ls.drainage[i,j]>0: #this cell is a drainage point (it drains a pool)
                
                    if (Z[i,j]>=Z[iR,j]) and ls.pool[iR,j]!=ls.drainage[i,j]: 
                        gradient = (Z[i,j]-Z[iR,j])/dx #pool is on my left, I drain to the right, use this gradiant
                    elif (Z[i,j]>=Z[iL,j]) and ls.pool[iL,j]!=ls.drainage[i,j]:
                        gradient = (Z[i,j]-Z[iL,j])/dx
                    elif (Z[i,j]>=Z[i,jU]) and ls.pool[i,jU]!=ls.drainage[i,j]:
                        gradient = (Z[i,j]-Z[i,jU])/dy
                    elif (Z[i,j]>=Z[i,jD]) and ls.pool[i,jD]!=ls.drainage[i,j]:
                        gradient = (Z[i,j]-Z[i,jD])/dy
                    else:
                        gradient = 0.02 # ??? This does happen (maybe when two pools merge)
                    Psi_z = K * ( ls.A[i,j]**m * gradient**n - theta_c)
                
                else: #this cell is a pool, assume it has some mass diffusion but no erosion!
                    Psi_z = 0.
                
                if (Psi_z<0):
                    Psi_z = 0. 

                # diffusion term
                Phi_z = D * ( (Z[iR,j] - 2.*Z[i,j] + Z[iL,j]) / dx**2 \
                            + (Z[i,jU] - 2.*Z[i,j] + Z[i,jD]) / dx**2 )
           
            Znew[i,j] = Z[i,j] + (Phi_z - Psi_z + uplift )*dt  

            dZdt= (Znew[i,j] - Z[i,j]) / dt
            CFL = abs(dZdt) * dt / min(dx,dy)
            if (CFL>1.0):
                print('\nWarning: Time step of',dt,'is probably too large. Safer would be:',dt/CFL)
            
            if (Znew[i,j]<0.):
                Znew[i,j] = 0. # yes, this does happen at the boundary when kept at zero
    
    #Znew[0,:] = 0.0 # resets front boundary to 0
    Z = np.copy(Znew)
    
    ls.pool_check(Z,NX,NY)

    if (np.mod(it,10)==0): 
        print(it,end='')
        update_figure()
        print(' Ocean level=',ls.ZBeachLevel,' Ocean surface fraction=',100*ls.AOcean/(NX*NY));
        if (100*ls.AOcean/(NX*NY)) == 100.0:
            break
    else:
        print('.',end='')

update_figure()
print(' Simulation finished.')
































































































.





























































































.





































































.



















































































.




















































































.















































































.



























































































.























































































.





































































.





































10 Ocean level= 3.7475047863663917  Ocean surface fraction= 83.0

















.



........20

/Users/CEE/anaconda3/lib/python3.6/site-packages/matplotlib/contour.py:1173: UserWarning: No contour levels were found within the data range.
  warnings.warn("No contour levels were found"


 Ocean level= 3.7475047863663917  Ocean surface fraction= 100.0
 Simulation finished.


In [10]:
for i in range(10000):
    h  = (0.1+rand())*5  #random 
    if h >= 5.5:
        print(h)

h is a random number between 0.5 and 5.5

In [11]:
print(NX)
for i in range(1000):
    r  = (0.1+rand())*NX
    if r <= 1:
        print(r)

10


r is a random value between 1 and 11

In [12]:
print(NY)
for i in range(100):
    yy = rand()*NY
    if yy <=1:
        print(yy)

10
0.1789287653807503
0.4125517456172456
0.6810427336917613
0.9707671374130056
0.1865646671291421
0.7898140938604525
0.9551883049844645


yy is a random value between 0 and 10

In [13]:
# for i in range(10000):
#     h  = (0.1+rand())*5
#     r  = (0.1+rand())*NX

#     for x in range(NX):
#         for y in range(NY):
#             dx = np.mod(x-xx+NX/2,NX)-NX/2; # difference i-i0 but apply p.b.c. 
#             dy = np.mod(y-yy+NY/2,NY)-NY/2;
#             dr = np.sqrt(dx**2+dy**2);
    
#     if h * (np.cos(dr/r*np.pi/2.0))**2 < 0.01:
#         if (dr<r):
#             print('h: ', h)
#             print('r: ', r)
#             print('dr: ', dr)
#             print((np.cos(dr/r*np.pi/2.0))**2)
#             print('value to be added to Z: ', h * (np.cos(dr/r*np.pi/2.0))**2)
                